In [ ]:

        # add a point of interpolate object annotation
        self.sam_interpolate_annotation_button = QtWidgets.QPushButton()
        self.sam_interpolate_annotation_button.setStyleSheet(
            "QPushButton { font-size: 10pt; font-weight: bold; }")
        self.sam_interpolate_annotation_button.setText("Interpolate Annotation")
        self.sam_interpolate_annotation_button.clicked.connect(
            self.sam_interpolate_annotation_button_clicked)
        self.sam_toolbar.addWidget(self.sam_interpolate_annotation_button)
        

    def sam_interpolate_annotation_button_clicked(self):
        x = self.INDEX_OF_CURRENT_FRAME
        self.sam_model_comboBox.setCurrentIndex(1)
        max_id = 0
        for i in range(1, x + 1, 10):
            self.main_video_frames_slider.setValue(i)
            self.main_video_frames_slider_changed()
            try:
                same_image = self.sam_predictor.check_image(
                    self.CURRENT_FRAME_IMAGE)
            except:
                return
            if not same_image:
                self.sam_clear_annotation_button_clicked()
                self.sam_buttons_colors("X")
            print(f'------------------------------------- now in frame {self.INDEX_OF_CURRENT_FRAME}-------------------------------------')
            for shapei in range(len(self.CURRENT_SHAPES_IN_IMG)):
                shape = self.CURRENT_SHAPES_IN_IMG[shapei]
                print(f'agjusting points of shape with id = {self.CURRENT_SHAPES_IN_IMG[shapei]["group_id"]}')
                x1, y1, x2, y2 = shape["bbox"]
                listPOINTS = [min(x1, x2), min(y1, y2), max(x1, x2), max(y1, y2)]
                listPOINTS = [int(round(x)) for x in listPOINTS]
                input_boxes = [listPOINTS]
                mask, score = self.sam_predictor.predict(point_coords=None,
                                                 point_labels=None,
                                                 box=input_boxes,
                                                 image=self.CURRENT_FRAME_IMAGE)
                points = self.sam_predictor.mask_to_polygons(mask)
                shape = self.sam_predictor.polygon_to_shape(points, score)
                self.CURRENT_SHAPES_IN_IMG[shapei]["points"] = shape["points"]
                id = self.CURRENT_SHAPES_IN_IMG[shapei]["group_id"]
                max_id = max(max_id, id)
                try:
                    self.key_frames['id_' +
                                    str(id)].append(self.INDEX_OF_CURRENT_FRAME)
                except:
                    self.key_frames['id_' +
                                str(id)] = [self.INDEX_OF_CURRENT_FRAME]
            print(f'updating shapes the updating current frame annotation')
            self.canvas.shapes = convert_shapes_to_qt_shapes(self.CURRENT_SHAPES_IN_IMG)
            self.update_current_frame_annotation()
            self.main_video_frames_slider_changed()
        
        for i in range(1, max_id + 1):
            print(f'interpolating id = {i}')
            try:
                x = self.key_frames['id_' +
                                    str(id)]
            except:
                continue
            self.interpolate(id=i, only_edited=True)




In [ ]:
def interpolate_with_sam(self, id):
    
        self.waitWindow(
            visible=True, text=f'Wait a second.\nIDs is being interpolated with SAM...')

        if self.sam_model_comboBox.currentText() == "Select Model (SAM disable)" or len(self.canvas.selectedShapes) == 0:
            return

        idsLIST = [shape.group_id for shape in self.canvas.selectedShapes]
        first_frame_idxLIST = [-1 for i in range(len(idsLIST))]
        last_frame_idxLIST = [-1 for i in range(len(idsLIST))]
        listObj = self.load_objects_from_json()
        for i in range(len(listObj)):
            self.waitWindow(visible=True)
            listobjframe = listObj[i]['frame_idx']
            frameobjects = listObj[i]['frame_data']
            for object_ in frameobjects:
                if (object_['tracker_id'] in idsLIST):
                    index = idsLIST.index(object_['tracker_id'])
                    first_frame_idxLIST[index] = min(
                        first_frame_idxLIST[index], listobjframe) if first_frame_idxLIST[index] != -1 else listobjframe
                    last_frame_idxLIST[index] = max(
                        last_frame_idxLIST[index], listobjframe) if last_frame_idxLIST[index] != -1 else listobjframe
        for i in range(len(idsLIST)):
            self.waitWindow(visible=True)
            if (first_frame_idxLIST[i] == -1 or last_frame_idxLIST[i] == -1 or first_frame_idxLIST[i] >= last_frame_idxLIST[i]):
                idsLIST.pop(i)
                first_frame_idxLIST.pop(i)
                last_frame_idxLIST.pop(i)
        if len(idsLIST) == 0:
            return

        recordsLIST = [[None for ii in range(
            first_frame_idxLIST[i], last_frame_idxLIST[i] + 1)] for i in range(len(idsLIST))]
        for i in range(len(listObj)):
            self.waitWindow(visible=True)
            listobjframe = listObj[i]['frame_idx']
            if (listobjframe < min(first_frame_idxLIST) or listobjframe > max(last_frame_idxLIST)):
                continue
            frameobjects = listObj[i]['frame_data']
            for object_ in frameobjects:
                if (object_['tracker_id'] in idsLIST):
                    index = idsLIST.index(object_['tracker_id'])
                    recordsLIST[index][listobjframe -
                                       first_frame_idxLIST[index]] = object_

        

        for frameIDX in range(min(first_frame_idxLIST), max(last_frame_idxLIST) + 1):
            self.waitWindow(visible=True)
            
            frameIMAGE = self.get_frame_by_idx(frameIDX)
            try:
                same_image = self.sam_predictor.check_image(
                    frameIMAGE)
            except:
                return

            for ididx in range(len(idsLIST)):
                i = frameIDX - first_frame_idxLIST[ididx]
                self.waitWindow(visible=True)
                if frameIDX < first_frame_idxLIST[ididx] or frameIDX > last_frame_idxLIST[ididx]:
                    continue
                
                records = recordsLIST[ididx]
                if (records[i] != None):
                    cur_bbox = records[frameIDX - first_frame_idxLIST[ididx]]['bbox']
                else:
                    prev_idx = i - 1
                    current = records[i - 1]
                    next_idx = i + 1
                    for j in range(i + 1, len(records)):
                        self.waitWindow(visible=True)
                        if (records[j] != None):
                            next_idx = j
                            break
                    cur_bbox = ((next_idx - i) / (next_idx - prev_idx)) * np.array(records[prev_idx]['bbox']) + (
                        (i - prev_idx) / (next_idx - prev_idx)) * np.array(records[next_idx]['bbox'])
                    cur_bbox = [int(cur_bbox[i]) for i in range(len(cur_bbox))]
                
                cur_bbox, cur_segment = self.sam_bbox_segment(frameIMAGE, cur_bbox, itr = 1)
                
                current['bbox'] = cur_bbox.copy()
                current['segment'] = cur_segment.copy()
                
                records[i] = current.copy()
                recordsLIST[ididx] = records

        for i in range(len(listObj)):
            self.waitWindow(visible=True)
            listobjframe = listObj[i]['frame_idx']
            if (listobjframe < min(first_frame_idxLIST) or listobjframe > max(last_frame_idxLIST)):
                continue
            for object_ in listObj[i]['frame_data']:
                if (object_['tracker_id'] in idsLIST):
                    listObj[i]['frame_data'].remove(object_)

            for ididx in range(len(idsLIST)):
                if (listobjframe < first_frame_idxLIST[ididx] or listobjframe > last_frame_idxLIST[ididx]):
                    continue
                idx = listobjframe - first_frame_idxLIST[ididx]
                idx = max(idx, 0)
                idx = min(idx, len(recordsLIST[ididx]) - 1)
                listObj[i]['frame_data'].append(recordsLIST[ididx][idx])

        self.waitWindow()
        self.load_objects_to_json(listObj)
        self.calc_trajectory_when_open_video()
        self.main_video_frames_slider_changed()
